In [ ]:
# This simple example shows how to predict the performance of 5 user-selected sectors.
# It uses linear equation to fit the short time historical data (15 days) and then uses the fitted function and today's data to predict next day's performance.
# The meaning of the prediction is the variety of the portion of the sector among 5 user-selected sectors

In [ ]:
import pickle
import numpy as np
from scipy.optimize import lsq_linear
from collections import defaultdict

In [ ]:
import akshare as ak
stock_industry_sina_df = ak.stock_sector_spot(indicator="新浪行业")
#stock_industry_sina_df

In [ ]:
#labels = stock_industry_sina_df['label']
#for i, sector in enumerate(stock_industry_sina_df['板块']):
#    label = labels[i]
    #print(label, sector)

In [ ]:
#gn_gf 光伏  gn_ndc 钠电池  gn_gtdc 固态电池   gn_gfgn 光伏概念   gn_xqzy 稀缺资源   gn_skgn 涉矿概念   gn_qcdz 汽车电子  gn_zjqrgn 机器人概念

In [ ]:
# User need to select 5 sectors of interest first.
#select_labels =['gn_gfgn', 'gn_xqzy', 'gn_zjqrgn', 'gn_qcdz', 'gn_ndc']

In [ ]:
#select_labels =['new_dlhy', 'new_dzqj', 'new_hbhy', 'new_ysjs', 'new_qczz']
select_labels = ['new_dzqj', 'new_hbhy', 'new_hghy', 'new_yqyb', 'new_ysjs']

In [ ]:
sector_to_symbols = dict()

for label in stock_industry_sina_df['label']:
    if label in select_labels:
        #print(label)
        stock_sector_detail_df = ak.stock_sector_detail(sector=label)
        #print(stock_sector_detail_df)
        symbols = stock_sector_detail_df['symbol']
        #print(symbols)
        #print("here")
        sector_to_symbols[label] = symbols

In [ ]:
#with open('sector_to_symbols.pickle', 'wb') as f:
#    pickle.dump(sector_to_symbols, f)

In [ ]:
with open('sector_to_symbols.pickle', 'rb') as f:
    sector_to_symbols = pickle.load(f)
#sector_to_symbols

In [ ]:
#sector_to_symbols

In [ ]:
sector_to_hist = dict()
symbol_to_stock_zh_a_daily_qfq_df = dict()
for sector, symbols in sector_to_symbols.items():
    if sector in select_labels:
        for symbol in symbols:
            try:
                stock_zh_a_daily_qfq_df = ak.stock_zh_a_daily(symbol=symbol, start_date="20220324", end_date='20220602', adjust="hfq")
                symbol_to_stock_zh_a_daily_qfq_df[symbol] = stock_zh_a_daily_qfq_df
            except:
                pass


In [ ]:
#with open('symbol_to_stock_zh_a_daily_qfq_df.pickle', 'wb') as f:
#     pickle.dump(symbol_to_stock_zh_a_daily_qfq_df, f)

In [ ]:
new_sector_to_symbols = dict()
for sector, symbols in sector_to_symbols.items():
    if sector in select_labels:
        new_sector_to_symbols[sector] = symbols

In [ ]:
#new_sector_to_symbols

In [ ]:
with open('symbol_to_stock_zh_a_daily_qfq_df.pickle', 'rb') as f:
    my_dog = pickle.load(f)
#my_dog

In [ ]:
#my_dog['sh600328']

In [ ]:
# obtain the historical total capital of each sector
sector_to_hist = dict()
symbol_to_stock_zh_a_daily_qfq_df = dict()
for sector, symbols in new_sector_to_symbols.items():
    total = defaultdict(lambda:np.float(0.))
    i = 0
    for symbol in symbols:
        try:
            #print(symbol)
            stock_zh_a_daily_qfq_df = my_dog[symbol]
            #print(stock_zh_a_daily_qfq_df)
            dates = stock_zh_a_daily_qfq_df['date']
            prices = stock_zh_a_daily_qfq_df['close']
            #print(np.isnan(price))
            #print(price)
            shares = stock_zh_a_daily_qfq_df['outstanding_share']
            #print(np.isnan(shares))
            #print(shares)
            
            for i, date in enumerate(dates):
                price = prices[i]
                share = shares[i]
                t = price*share
                total[date] += t
            
        except:
            pass
        #print(np.isnan(total))

    sector_to_hist[sector] = total

In [ ]:
#sector_to_hist

In [ ]:
# calculate the total capital of all the sectors
total = defaultdict(lambda:np.float(0.))
for sector, hist in sector_to_hist.items():
    for date, t in hist.items():
        total[date] += t

In [ ]:
#total

In [ ]:
# calculate the historical portion of each sector
sector_to_portion = dict()
for sector, hist in sector_to_hist.items():
    portion = defaultdict(lambda:np.float(0.)) 
    for date, t in hist.items():
        portion[date] = t/total[date]
    sector_to_portion[sector] = portion

In [ ]:
#sector_to_portion

In [ ]:
# check the sum is one
sum_check = defaultdict(lambda:np.float(0.))
for sector, portions in sector_to_portion.items():
    for date, p in portions.items():
        sum_check[date] += p

In [ ]:
sum_check

In [ ]:
# calculate the historical protion variation
sector_to_dt_portion = defaultdict(lambda:np.float(0.))
for sector, portions in sector_to_portion.items():
    #print(portions)
    i = 0
    dt_portion = defaultdict(lambda:np.float(0.))
    for date, p in portions.items():
        if i == 0:
            last_portion = p
            i += 1
        else:
            dt_portion[date] = p - last_portion
            last_portion = p
    #print(dt_portion)
    sector_to_dt_portion[sector] = dt_portion

In [ ]:
len(sector_to_dt_portion['new_dzqj'])

In [ ]:
x = np.zeros((45, 5))
y = np.zeros((5, 45))
j = 0
for sector, dt_portions in sector_to_dt_portion.items():
    i = 0
    for date, dt_portion in dt_portions.items():
        x[i][j] = dt_portion
        y[j][i] = dt_portion
        i += 1
    j += 1
        
        

In [ ]:
#x

In [ ]:
y

In [ ]:
import numpy as np
from scipy import optimize		# 最小二乘法拟合
import matplotlib.pyplot as plt 	# python matplotlib 绘图
from mpl_toolkits.mplot3d import Axes3D  # 3D 绘图


def func(x_list, p):
    """ 数据拟合所用的函数：z=p*x
    :param x: 自变量 x
    :param p: 拟合参数
    """
    f = np.tensordot(x_list, p[:-1], axes = 1) + p[-1]
    #print(x_list)
    #print(y)
    return f


def residuals(p, z, x_list):
    """ 得到数据 z 和拟合函数之间的差
    """
    #print("z: ", z)
    #print(func(x_list, p))
    return z - func(x_list, p)

def cost(p, z, x_list):
    r = residuals(p, z, x_list)
    r = r**2
    return r.sum()


def gridsearch(z, x_list):
    least = np.inf
    best_p = None
    p = np.zeros([6])
    s = [i/10. for i in range(-10, 11)]
    #s = [0.2, 1.]
    
    for p0 in s:
        p[0] = p0
        
        for p1 in s:
            p[1] = p1
            
            for p2 in s:
                p[2] = p2
            
                for p3 in s:
                    p[3] = p3
                    
                    for p4 in s:
                        p[4] = p4
                        
                        for p5 in s:
                            p[5] = p5
                    
                        
                            lst = cost(p, z, x_list)
                            #print(p, lst)
                        
                            if lst < least:
                                #print(p, lst)
                                least = lst
                                best_p = list(p)
    print(least)
    print(best_p)
    return np.array(best_p)
    
def predict(x, best_p):
    #print(best_p)
    #print(x)
    predict = np.tensordot(x, best_p[:-1], axes = 1) + best_p[-1]
    return predict

In [ ]:
from copy import deepcopy
lb = [-1., -1., -1., -1., -1.]
lu = [1., 1., 1., 1., 1.]
def accuracy():
    correct = 0
    total = 0
    best_p = np.zeros([6])
    sum_red = 0.
    count_purchase = 0
    #print(best_p)
    for i in range(29):
        
        max_pred = -10000000.
        sector = 100000
        for s in range(5):  # 5 sectors
            #print(i)
            n_y = y[s][1+i:16+i]
            n_x = x[i:15+i]
            dc_n_y = deepcopy(n_y)
            dc_n_x = deepcopy(n_x)
            #best_p = gridsearch(n_y, n_x)
            result = lsq_linear(dc_n_x, dc_n_y, bounds = (lb, lu), max_iter = 1000000)
            coef = deepcopy(result['x'])
            for idx, p in enumerate(coef):  # be carefull!! cannot use i, otherwise will change i value in the first loop
                best_p[idx] = p
            t_x = x[15+i:16+i]
            pred = predict(t_x, best_p)
            if pred > max_pred:
                max_pred = pred
                sector = s
            
            
            
            target = y[s][16+i:17+i]
            judge = pred*target
            print(pred, target, judge)
            #print(y[s])
            #print(' ')
            if 1:#pred[0] > 0.00:  # only see strong prediction
                if judge > 0:
                    correct += 1
                total += 1
            if pred[0] > 0.:
                sum_red += target
                count_purchase += 1
        print(' ')
    print("accuracy is:  ", correct/total)
    print("expected benefit per purchase: ", sum_red/count_purchase)

In [ ]:
#p = np.array([0.98779583, -0.99550428,  0.48027421,  0.6813524,   0.28157936,  0.])
#predict(np.array([[-0.00692854, -0.00021002, -0.00277886,  0.00942962,  0.0004878 ]]), p)

In [ ]:
# prediction, really, prediction*really
accuracy()

In [ ]:
lb = [-1., -1., -1., -1., -1.]
lu = [1., 1., 1., 1., 1.]
best_p = np.zeros([6])
n_y = y[0][-16:-1]
n_x = x[-17:-2]
#best_p = gridsearch(n_y, n_x)
result = lsq_linear(n_x, n_y, bounds = (lb, lu), max_iter = 1000000)
for idx, p in enumerate(result['x']):
    best_p[idx] = p
t_x = x[-2:-1]
pred = predict(t_x, best_p)

In [ ]:
pred

In [ ]:
y[0][-1]

In [ ]:
for i in range(5):
    lb = [-1., -1., -1., -1., -1.]
    lu = [1., 1., 1., 1., 1.]
    best_p = np.zeros([6])
    n_y = y[i][-15:]
    n_x = x[-16:-1]
    #best_p = gridsearch(n_y, n_x)
    result = lsq_linear(n_x, n_y, bounds = (lb, lu), max_iter = 1000000)
    for idx, p in enumerate(result['x']):
        best_p[idx] = p
    t_x = x[-1:]
    pred = predict(t_x, best_p)
    print(pred)